In [ ]:
# MeshCat.jl

This notebook demonstrates interacting with the MeshCat viewer from Julia. You'll need to have installed MeshCat.jl from <https://github.com/rdeits/MeshCat.jl>

In [1]:
# Optional: 
# These commands tell the Julia package manager to use the exact
# set of dependencies specified in the Project.toml file in this folder. 
# That should give you a nice, reproducible environment for testing. 

using Pkg
Pkg.activate(@__DIR__)
Pkg.develop(Pkg.PackageSpec(path=dirname(@__DIR__)))
Pkg.instantiate()

 Activating environment at `d:\Users\Personal\Personal\Projects\GitRepo\LearnJulia\Ex\Project.toml`


LoadError: could not find project file in package at `d:\Users\Personal\Personal\Projects\GitRepo\LearnJulia` maybe `subdir` needs to be specified

In [6]:
# Activate the MeshCat package, and import some other 
# useful functions
using MeshCat
using CoordinateTransformations
using Rotations
using GeometryBasics: HyperRectangle, Vec, Point, Mesh
using Colors: RGBA, RGB

In [7]:
# Create a new visualizer instance
vis = Visualizer()

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat C:\Users\dengy\.julia\packages\MeshCat\DjQfQ\src\visualizer.jl:73


MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [8]:
# The `render(vis)` function will try to render a MeshCat
# visualizer inline. In Jupyter, it will cause the visualizer
# to be displayed in the output of the current Jupyter cell. 
# In Juno/VSCode, it will cause the visualizer to be displayed in
# the plot pane.
render(vis)

MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String,Array{UInt8,1}}(), nothing, Dict{String,MeshCat.SceneTrees.SceneNode}()), Set{Any}(), ip"127.0.0.1", 8700))

There are several ways you can open the visualizer if you like:

### In-Browser
To open the meshcat viewer in your browser, you can simply do:

```julia
open(vis)
```

### Standalone (with Blink.jl)

Blink.jl allows you to create standalone web apps with Electron. To use
Blink with MeshCat, you need to install Blink:

```julia
Pkg.add("Blink")
using Blink
Blink.AtomShell.install()
```

Then you can open MeshCat in a Blink window with:

```julia
open(vis, Blink.Window())
```

### Embed the visualizer inside this notebook

```julia
render(vis)
```

In [9]:
# We also need to check if this notebook is running on Travis CI
# since the browser doesn't work properly there. You shouldn't 
# need this in your code, and you can just do `open(vis)`.

if !haskey(ENV, "CI")
    open(vis)
end

# To block execution until the visualizer is open, you can do:
# wait(vis)

Process(`cmd.exe /C 'start http://127.0.0.1:8700'`, ProcessExited(0))

In [10]:
# First, we'll create a simple geometric object
box = HyperRectangle(Vec(0., 0, 0), Vec(1., 1, 1))

HyperRectangle{3,Float64}([0.0, 0.0, 0.0], [1.0, 1.0, 1.0])

In [12]:
# setobject! causes the visualizer to render the given geometry. 
# Initially that geometry will be placed in the zero configuration
# (that is, its position and rotation are zero)
setobject!(vis, box)

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [13]:
# We can use settransform!() to tell the viewer to draw the box at a specific
# position. Translation() creates a CoordinateTransformations.Transformation
# corresponding to the given x; y; z translation.
settransform!(vis, Translation(0., 1, 0))

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [14]:
# We can also rotate the model by sending a different transformation
settransform!(vis, LinearMap(AngleAxis(pi/4, 0, 0, 1)))

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [15]:
# And we can clear the box
delete!(vis)

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [16]:
# Next, let's add the same box, but color it green this time.
green_material = MeshPhongMaterial(color=RGBA(0, 1, 0, 0.5))
setobject!(vis, box, green_material)

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700

In [17]:
delete!(vis)

MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700


# The Viewer Tree

Now that we've introduced the basics of loading and transforming geometries, it's time to introduce the tree model used by MeshCat.jl. The idea is pretty simple:

* A Visualizer contains a tree of geometries
* We add, draw, and delete geometries by using their path, which is the path from the root of the tree to that geometry
* We can set the transform for any node in the tree. Setting the transform for a node in the tree affects all of its descendant geometries.



In [18]:
# We can access a particular path within the visualizer with indexing notation:
vis["group1"]

MeshCat Visualizer with path /meshcat/group1 at http://127.0.0.1:8700

In [19]:
# We load geometries using the same path notation:
green_box_vis = setobject!(vis["group1"]["greenbox"], box, green_material)

MeshCat Visualizer with path /meshcat/group1/greenbox at http://127.0.0.1:8700

Click "open controls" in the MeshCat viewer. You should see a `meshcat` folder, which you can click to expand:

    Scene
      | meshcat
          | group1
              | greenbox

  

In [20]:
# And we set transforms in the same way:
settransform!(green_box_vis, Translation(0, 0, 1))

MeshCat Visualizer with path /meshcat/group1/greenbox at http://127.0.0.1:8700

In [21]:
# The same indexing notation makes it easy to get a handle to a 
# particular part of the viewer tree:
group1 = vis["group1"]
settransform!(group1, Translation(0, 0, -1))

MeshCat Visualizer with path /meshcat/group1 at http://127.0.0.1:8700

The viewer tree becomes much more interesting (and useful) when we have multiple geometries. Let's add another geometry in the same group:

In [22]:
blue_material = MeshPhongMaterial(color=RGBA(0, 0, 1, 0.5))
setobject!(vis["group1"]["bluebox"], box, blue_material)

MeshCat Visualizer with path /meshcat/group1/bluebox at http://127.0.0.1:8700

The scene browser will now show:

    Scene
      | meshcat
          | group1
              | greenbox
              | bluebox

Since both the green and blue box are inside `group1`, we can move them together by transforming that whole group:

In [23]:
settransform!(vis["group1"], Translation(1, 0, 0))

MeshCat Visualizer with path /meshcat/group1 at http://127.0.0.1:8700

Or we can move just the green box:

In [24]:
settransform!(vis["group1"]["greenbox"], Translation(0, 1, 0))

MeshCat Visualizer with path /meshcat/group1/greenbox at http://127.0.0.1:8700

The final pose of each geometry in the tree is just the composition of all of the transforms in the path from the root of the tree to that geometry. 

Likewise, we can delete an entire group:

In [25]:
delete!(vis["group1"])

MeshCat Visualizer with path /meshcat/group1 at http://127.0.0.1:8700

In [26]:
# Of course, we can draw much more interesting geometries than 
# just simple boxes. Let's load a 3D mesh and visualize it:
using MeshIO
using FileIO
cat_mesh = load(joinpath(dirname(pathof(MeshCat)), "..", "test", "data", "meshes",
        "cat.obj"))
setobject!(vis, cat_mesh)
settransform!(vis, LinearMap(AngleAxis(pi/2, 1, 0, 0)))

LoadError: ArgumentError: Package MeshIO not found in current path:
- Run `import Pkg; Pkg.add("MeshIO")` to install the MeshIO package.


In [27]:
delete!(vis)

# Next, let's create a triangulated mesh by finding
# the 0-level set of some function. 
# 
# First, we'll define our function:
f = x -> sum(sin, 5 * x)

# Then we pick a region of interest in which to sample the function.
# This region starts at (-1, -1, -1) and extends to (1, 1, 1):
lower_bound = Vec(-1.,-1,-1)
upper_bound = Vec(1., 1, 1)

# Those two pieces of information are all we need to construct a mesh geometry.
#
# Using `Meshing.jl`, we can construct a mesh directly from our function:
using Meshing: MarchingCubes

bounds = HyperRectangle(lower_bound, upper_bound - lower_bound)
mesh = Mesh(f, bounds, MarchingCubes())

# # geometry. First, we sample the function f using GeometryTypes.SignedDistanceField:

# sdf = SignedDistanceField(f, bounds)

# # Now we use the Marching Tetrahedra algorithm to find a mesh describing
# # the surface for which f(x) == 0
# using Meshing: MarchingTetrahedra

# mesh = HomogenousMesh(sdf, MarchingTetrahedra())

# And now we can load that geometry into the visualizer
setobject!(vis, mesh)

LoadError: ArgumentError: Package Meshing not found in current path:
- Run `import Pkg; Pkg.add("Meshing")` to install the Meshing package.


# More Geometry Types

`MeshCat.jl` supports a few extra types of geometry that you may want to visualize. One such geometry is a point cloud: 

## Point Clouds

A PointCloud can be constructed from a vector of points. The easiest way to represent a point is just a standard Julia vector:

In [28]:
delete!(vis)
pointcloud = PointCloud([[x, 0 + 0.01 * randn(), 0.5] for x in range(-1, stop=1, length=1000)])
setobject!(vis[:pointcloud], pointcloud)

MeshCat Visualizer with path /meshcat/pointcloud at http://127.0.0.1:8700

For a much more efficient point cloud, try using a vector of `StaticVector`s from StaticArrays.jl. The `Point3f0` is one such static vector:


In [29]:
using GeometryBasics: Point3f0
points = rand(Point3f0, 1000000)
setobject!(vis[:pointcloud], PointCloud(points))

MeshCat Visualizer with path /meshcat/pointcloud at http://127.0.0.1:8700

You can also color each point by passing a vector of `RGB{Float32}` elements to `PointCloud`:

In [30]:
colors = reinterpret(RGB{Float32}, points); # use the xyz value as rgb color
setobject!(vis[:pointcloud], PointCloud(points, colors))

MeshCat Visualizer with path /meshcat/pointcloud at http://127.0.0.1:8700

## Textured Meshes

Meshes can be textured by passing a `Texture` parameter to the material constructor. Currently, the only supported texture type is `PngImage`, which simply transmits the data from an image as base64-encoded PNG data. 

In [31]:
image = PngImage(
    joinpath(MeshCat.VIEWER_ROOT, "..", "data", "HeadTextureMultisense.png"))
texture = Texture(image=image)
material = MeshLambertMaterial(map=texture)
geometry = load(joinpath(MeshCat.VIEWER_ROOT, "..", "data", "head_multisense.obj"))
setobject!(vis["robots", "valkyrie", "head"], geometry, material)
settransform!(vis["robots", "valkyrie"], Translation(0.5, -0.5, 0.5))

LoadError: UndefVarError: load not defined

## Mesh Files

While MeshCat should be able to render any mesh you can load as any kind of `GeometryBasics.AbstractMesh`, there are a wide variety of mesh file formas, and not all of them can be easily loaded in Julia right now. For OBJ, DAE, and STL (ASCII and binary) meshes, there is an additional option: The `MeshFileGeometry` type passes a raw mesh file directly to the MeshCat viewer, letting the existing mesh decoders built into that viewer take care of loading the data for you. For example: 

In [32]:
path = joinpath(@__DIR__, "..", "test", "data", "meshes", "cube.dae")
setobject!(vis["meshes", "dae_file_geometry"], 
    MeshFileGeometry(path))
settransform!(vis["meshes", "dae_file_geometry"], 
    Translation(0.0, 1.25, 0.0))

LoadError: SystemError: opening file "d:\\Users\\Personal\\Personal\\Projects\\GitRepo\\LearnJulia\\Ex\\..\\test\\data\\meshes\\cube.dae": No such file or directory

Note that in this case only the *geometry* of the mesh is loaded. 

# Mesh File Objects

Certain kinds of meshes, like OBJ and DAE files, can contain meshes with associated textures or even multiple such objects in a single file. To render a mesh file, including all of its associated materials and textures, you can use the `MeshFileObject` type, which passes the entire mesh file contents *and its associated materials* to the MeshCat viewer, again letting the built-in decoders handle all of the data loading.

In [33]:
path = joinpath(@__DIR__, "..", "test", "data", "meshes", "cube.dae")

setobject!(vis["meshes", "dae_file_object"], 
    MeshFileObject(path))
settransform!(vis["meshes", "dae_file_object"], 
    Translation(0.0, 2.5, 0.0))

LoadError: SystemError: opening file "d:\\Users\\Personal\\Personal\\Projects\\GitRepo\\LearnJulia\\Ex\\..\\test\\data\\meshes\\cube.dae": No such file or directory